<a href="https://colab.research.google.com/github/aithaprasad/related-languages/blob/main/NLP_closely_related_langs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_source = open('train-source.txt', 'r')

In [3]:
train_source_sentences = []
temp = []
for line in train_source:
  token = line.rstrip("\n")
  if token == '<s>': temp = []
  elif token == '</s>': train_source_sentences.append(temp)
  else: temp.append(line)

In [4]:
train_target = open('train-target.txt', 'r')

In [5]:
train_target_sentences = []
temp = []
for line in train_target:
  token = line.rstrip("\n")
  if token == '<s>': temp = []
  elif token == '</s>': train_target_sentences.append(temp)
  else: temp.append(line)

In [6]:
len(train_source_sentences) == len(train_target_sentences) 

True

In [7]:
from keras.preprocessing.text import Tokenizer
def tokenization(lines):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

In [8]:
source_tokenizer = tokenization(train_source_sentences)
source_vocab_size = len(source_tokenizer.word_index) + 1

In [9]:
mini, maxi = 0, 0
for sent in train_source_sentences:
  sizes = len(sent)
  if len(sent) <= 25: mini += 1
  else: maxi += 1
print(mini)
print(maxi)

35761
9410


In [10]:
source_max_length = 25
print('Source Vocabulary Size: %d' % source_vocab_size)

Source Vocabulary Size: 31038


In [11]:
target_tokenizer = tokenization(train_target_sentences)
target_vocab_size = len(target_tokenizer.word_index) + 1
target_max_length = 25
print('Target Vocabulary Size: %d' % target_vocab_size)

Target Vocabulary Size: 25738


In [12]:
from keras_preprocessing.sequence import pad_sequences
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    seq = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

In [13]:
from sklearn.model_selection import train_test_split

# split data into train and test set
source_train, source_test = train_test_split(train_source_sentences, test_size=0.2, random_state = 42)
target_train, target_test = train_test_split(train_target_sentences, test_size=0.2, random_state = 42)

In [14]:
# prepare training data
trainX = encode_sequences(source_tokenizer, source_max_length, source_train)
trainY = encode_sequences(target_tokenizer, target_max_length, target_train)

# prepare validation data
testX = encode_sequences(source_tokenizer, source_max_length, source_test)
testY = encode_sequences(target_tokenizer, target_max_length, target_test)

In [15]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
# build NMT model
def define_model(in_vocab,out_vocab, in_timesteps,out_timesteps,units):
    model = Sequential()
    model.add(Embedding(in_vocab, units, input_length=in_timesteps, mask_zero=True))
    model.add(LSTM(units))
    model.add(RepeatVector(out_timesteps))
    model.add(LSTM(units, return_sequences=True))
    model.add(Dense(out_vocab, activation='softmax'))
    return model

In [16]:
model = define_model(source_vocab_size, target_vocab_size, source_max_length, target_max_length, 512)

In [17]:
from keras import optimizers
rms = optimizers.RMSprop(lr=0.1)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy')

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [18]:
history = model.fit(trainX, trainY.reshape(trainY.shape[0], trainY.shape[1], 1),
                    epochs=2, batch_size=512, validation_split = 0.2, 
                    verbose=1)

Epoch 1/2
57/57 [==============================] - 1863s 33s/step - loss: 10.7829 - val_loss: 5.2710
Epoch 2/2
57/57 [==============================] - 1862s 33s/step - loss: 5.0850 - val_loss: 5.2928


In [ ]:
preds = model.predict(testX.reshape((testX.shape[0],testX.shape[1])))

136/283 [=============>................] - ETA: 1:56

In [ ]:
def get_word(n, tokenizer):
  for word, index in tokenizer.word_index.items():
    if index == n: return word
  return None